In [4]:
import os
import sys

project_root = os.getcwd()
while "src" not in os.listdir(project_root):
    project_root = os.path.dirname(project_root)
sys.path.append(project_root)

import torch
from src.models import DeepSampler
from src.utils.train.losses import MultiSourceLoss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Audio parameters
SR = 44100
NFFT = 2048
HOP = 512
CHUNK_DUR = 2
OVERLAP = 0
BATCH_SIZE = 4
IN_CHANNELS = 1
OUT_CHANNELS = 4

In [5]:
rand_input = torch.rand(
    BATCH_SIZE, IN_CHANNELS, NFFT // 2 + 1, int(CHUNK_DUR * SR) // HOP + 1
).to(device)
rand_target = torch.rand(
    BATCH_SIZE, OUT_CHANNELS, NFFT // 2 + 1, int(CHUNK_DUR * SR) // HOP + 1
).to(device)
model = DeepSampler(debug=True).to(device)
# Test forward pass
output = model(rand_input)
output.shape

DINOs: input shape: torch.Size([4, 1, 1025, 173])
DINOs: init_processed shape: torch.Size([4, 16, 1025, 173])
DINOs: freq_attention shape: torch.Size([4, 16, 1025, 173])
Encoder(16->32): input shape: torch.Size([4, 16, 1025, 173])
Encoder(16->32): after_conv shape: torch.Size([4, 32, 1025, 173])
Encoder(16->32): after_norm shape: torch.Size([4, 32, 1025, 173])
Encoder(16->32): after_act shape: torch.Size([4, 32, 1025, 173])
Encoder(16->32): after_pool shape: torch.Size([4, 32, 512, 86])
Encoder(16->32): after_dropout shape: torch.Size([4, 32, 512, 86])
DINOs: enc_1_out shape: torch.Size([4, 32, 512, 86])
Encoder(32->64): input shape: torch.Size([4, 32, 512, 86])
Encoder(32->64): after_conv shape: torch.Size([4, 64, 512, 86])
Encoder(32->64): after_norm shape: torch.Size([4, 64, 512, 86])
Encoder(32->64): after_act shape: torch.Size([4, 64, 512, 86])
Encoder(32->64): after_pool shape: torch.Size([4, 64, 256, 43])
Encoder(32->64): after_dropout shape: torch.Size([4, 64, 256, 43])
DINOs: 

torch.Size([4, 4, 1025, 173])

In [6]:
# Test loss
loss_fn = MultiSourceLoss([1, 1, 1, 1]).to(device)
loss = loss_fn(output, rand_target)
print(loss)
loss.backward()

tensor(0.6045, device='cuda:0', grad_fn=<AddBackward0>)
